In [0]:
#!pip install tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 93kB/s 
     |████████████████████████████████| 419kB 39.4MB/s 
     |████████████████████████████████| 3.0MB 27.5MB/s 


In [0]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2.0.0-alpha0


In [0]:
# 구글드라이브에 폴더를 업로드하고 마운트
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
#확인코드
!ls /gdrive/My\ Drive/car/

test  train


In [0]:
#1. 데이터 생성하기
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 10, width_shift_range = 0.2, height_shift_range=0.2, shear_range = 0.7, zoom_range = [0.9,2.2], horizontal_flip=True, vertical_flip=True, fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(
    '/gdrive/My Drive/car/train',
    target_size=(36, 36),
    batch_size = 3,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
    '/gdrive/My Drive/car/test',
    target_size=(36,36),
    batch_size = 3,
    class_mode='categorical')

Found 122 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [0]:
from tensorflow.keras.layers import Dropout
#2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(36,36,3)))
model.add(Conv2D(62, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(62, (3,3), activation = 'relu'))
model.add(Conv2D(62, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation = 'softmax'))

#3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [0]:

#from tensorflow.keras.callbacks import EarlyStopping
#early_stopping=EarlyStopping(patience=20) # 조기종료 콜백함수 정의
#hist = model.fit(x_train, y_train, epochs=50, batch_size=10, verbose=1, validation_data=(x_val, y_val), callbacks=[early_stopping])
#4. 모델 학습시키기
model.fit_generator(
  train_generator,
  steps_per_epoch=50,
  epochs=200,
  validation_data=test_generator,
  validation_steps=5)

#5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))



Epoch 1/200
50/50 [==============================] - 54s 1s/step - loss: 1.6400 - accuracy: 0.2635 - val_loss: 1.6132 - val_accuracy: 0.1333
Epoch 2/200
50/50 [==============================] - 1s 17ms/step - loss: 1.5693 - accuracy: 0.2867 - val_loss: 1.6418 - val_accuracy: 0.1333
Epoch 3/200
50/50 [==============================] - 1s 19ms/step - loss: 1.4933 - accuracy: 0.3176 - val_loss: 1.6217 - val_accuracy: 0.1333
Epoch 4/200
50/50 [==============================] - 1s 18ms/step - loss: 1.5311 - accuracy: 0.3356 - val_loss: 1.6777 - val_accuracy: 0.1333
Epoch 5/200
50/50 [==============================] - 1s 19ms/step - loss: 1.4703 - accuracy: 0.3624 - val_loss: 1.7646 - val_accuracy: 0.2000
Epoch 6/200
50/50 [==============================] - 1s 18ms/step - loss: 1.5703 - accuracy: 0.3311 - val_loss: 1.5902 - val_accuracy: 0.2667
Epoch 7/200
50/50 [==============================] - 1s 18ms/step - loss: 1.4977 - accuracy: 0.3893 - val_loss: 1.7820 - val_accuracy: 0.1333
Epoch 8

In [0]:
#6. 모델 사용하기
print("-- Predict -- ")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict -- 
{'bus': 0, 'sedan': 1, 'sportscar': 2, 'suv': 3, 'truck': 4}
[[0.338 0.324 0.339]
 [0.345 0.305 0.350]
 [0.346 0.322 0.332]
 [0.352 0.323 0.325]
 [0.357 0.307 0.336]
 [0.373 0.304 0.323]
 [0.355 0.317 0.327]
 [0.347 0.323 0.330]
 [0.358 0.298 0.343]
 [0.319 0.313 0.368]
 [0.362 0.299 0.339]
 [0.336 0.337 0.327]
 [0.342 0.328 0.330]
 [0.349 0.316 0.335]
 [0.361 0.310 0.330]]
